In [1]:
import numpy as np

from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [4]:
cancer_data = load_breast_cancer()

x_data = cancer_data.data
y_label = cancer_data.target

X_train, X_test, y_train, y_test = train_test_split(x_data, y_label, test_size = 0.2, random_state = 2023)

In [3]:
knn_clf  = KNeighborsClassifier(n_neighbors=4)
rf_clf = RandomForestClassifier(n_estimators=100, random_state=0)
dt_clf = DecisionTreeClassifier()
ada_clf = AdaBoostClassifier(n_estimators=100)
lr_final = LogisticRegression(C=10)

In [5]:
knn_clf.fit(X_train, y_train)
rf_clf.fit(X_train , y_train)
dt_clf.fit(X_train , y_train)
ada_clf.fit(X_train, y_train)

AdaBoostClassifier(n_estimators=100)

## 개별 모델의 예측 정확도는?

In [6]:
knn_pred = knn_clf.predict(X_test)
rf_pred = rf_clf.predict(X_test)
dt_pred = dt_clf.predict(X_test)
ada_pred = ada_clf.predict(X_test)

print('KNN 정확도: {0:.4f}'.format(accuracy_score(y_test, knn_pred)))
print('랜덤 포레스트 정확도: {0:.4f}'.format(accuracy_score(y_test, rf_pred)))
print('결정 트리 정확도: {0:.4f}'.format(accuracy_score(y_test, dt_pred)))
print('에이다부스트 정확도: {0:.4f} :'.format(accuracy_score(y_test, ada_pred)))

KNN 정확도: 0.9386
랜덤 포레스트 정확도: 0.9561
결정 트리 정확도: 0.9474
에이다부스트 정확도: 0.9649 :


### 결과 데이터들을 합치기

In [7]:
pred= np.array([knn_pred, rf_pred, dt_pred, ada_pred])

pred = np.transpose(pred)
print(pred.shape)

(114, 4)


In [8]:
lr_final.fit(pred, y_test)
final = lr_final.predict(pred)

# Making K-Fold classifier

In [9]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error

In [10]:
def get_stacking_base_datasets(model, X_train_n, y_train_n, X_test_n, n_folds):
  kf = KFold(n_splits = n_folds, shuffle = False)
  train_fold_pred = np.zeros((X_train_n.shape[0], 1))
  test_pred = np.zeros((X_test_n.shape[0], n_folds))

  for folder_counter, (train_index, valid_index) in enumerate(kf.split(X_train_n)):
    print(f'폴드 세트 {folder_counter} 시작')
    X_tr = X_train_n[train_index]
    y_tr = y_train_n[train_index]
    X_te = X_train_n[valid_index]

    model.fit(X_tr, y_tr)
    # valid_index + index number
    train_fold_pred[valid_index, :] = model.predict(X_te).reshape(-1,1)
    # 본 X_test 예측
    test_pred[:, folder_counter] = model.predict(X_test_n)

  test_pred_mean = np.mean(test_pred, axis =1).reshape(-1,1)

  return train_fold_pred, test_pred_mean

In [11]:
knn_train, knn_test = get_stacking_base_datasets(knn_clf, X_train, y_train, X_test, 7)
rf_train, rf_test = get_stacking_base_datasets(rf_clf, X_train, y_train, X_test, 7)
dt_train, dt_test = get_stacking_base_datasets(dt_clf, X_train, y_train, X_test,  7)
ada_train, ada_test = get_stacking_base_datasets(ada_clf, X_train, y_train, X_test, 7)

폴드 세트 0 시작
폴드 세트 1 시작
폴드 세트 2 시작
폴드 세트 3 시작
폴드 세트 4 시작
폴드 세트 5 시작
폴드 세트 6 시작
폴드 세트 0 시작
폴드 세트 1 시작
폴드 세트 2 시작
폴드 세트 3 시작
폴드 세트 4 시작
폴드 세트 5 시작
폴드 세트 6 시작
폴드 세트 0 시작
폴드 세트 1 시작
폴드 세트 2 시작
폴드 세트 3 시작
폴드 세트 4 시작
폴드 세트 5 시작
폴드 세트 6 시작
폴드 세트 0 시작
폴드 세트 1 시작
폴드 세트 2 시작
폴드 세트 3 시작
폴드 세트 4 시작
폴드 세트 5 시작
폴드 세트 6 시작


In [13]:
Stacking_X_train = np.concatenate((knn_train, rf_train, dt_train, ada_train), axis=1)
Stacking_X_test = np.concatenate((knn_test, rf_test, dt_test, ada_test), axis = 1 )

In [14]:
lr_final.fit(Stacking_X_train, y_train)
stack_final = lr_final.predict(Stacking_X_test)

print(f'예측 정확도 {round(accuracy_score(y_test,stack_final),4)}')

예측 정확도 0.9649


스태킹을 이룰 때는 개별 모델을 최적으로 파라미터 튜닝 후 스태킹

메타 모델을 이용한 학습 데이터 셋, 예측 데이터 셋을 스태킹 형태로 결합